针对上述任意样本，比如SGI75460067的count/tpm/cpm，会得到如下的结果
|基因名称|	Hisat-rf结果|	Hisat-fr结果|	Star结果|
|-|-|-|-|
|ABC	|150	|140	|600|
|DEF	|120000	|10000	|2600|
|GHI	|0	|1|200|

算了，算了，还是手动合并算了

In [99]:
# import os
# import pandas as pd
# #"D:\\githubku\\my_jupyter\\20230206_tpm_count\\fina/12"文件夹下所有文件夹名，保存为list
# path = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/12"
# file_list = os.listdir(path)
# # 读取D:\\githubku\\my_jupyter\\20230206_tpm_count\\final中的count_all.xlsx，cpm_all.xlsx，tpm_all.xlsx
# count_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/count_all.xlsx")
# cpm_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/cpm_all.xlsx")
# tpm_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/tpm_all.xlsx")

In [1]:
import pandas as pd
import os
ref = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\/merge1v1.txt",sep="\t")
##每种类型所包含的基因名称，以及

C:\Users\zhangyifan1\AppData\Local\Temp\ipykernel_23000\1681191210.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ref = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\/merge1v1.txt",sep="\t")


In [101]:
id="105"

In [102]:
inputpath1 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\final\\12\\type\\tpm\\"+id+"\\"+ id+ ".txt"
outputpath1="D:\\githubku\\my_jupyter\\20230206_tpm_count\\final\\12\\type\\tpm\\fianltype\\"+id+"\\"

In [103]:
os.makedirs(os.path.dirname(outputpath1), exist_ok=True)

In [104]:
#保留Gene Name	Gene type列
ref = ref[["Gene Name","Gene type"]]
#去除gene name重复行
ref = ref.drop_duplicates(subset="Gene Name",keep="first")

In [105]:
#读入D:\githubku\my_jupyter\20230206_tpm_count\final\12\SGI75460067\0067count.txt文件
count = pd.read_csv(inputpath1,sep="\t")

In [106]:
#ref和count合并，以gene name为键，保留ref中的gene type
count1 = pd.merge(ref,count,on="Gene Name",how="right")


In [107]:
#将count1中的gene type列，转换为行索引
count1 = count1.set_index("Gene type")


In [ ]:
#输出count1中gene type为空的行
count1.loc[count1.index.isnull()]

In [110]:
#获取count1中索引，保存为list，去重
index_name = list(set(count1.index))
#遍历index_name,获取count1中对应索引的数据，保存到文件中
for i in index_name:
    count1[count1.index==i].to_csv(outputpath1+"\\"+id+".%s.txt"%i,sep="\t")


## 计算pearson相关系数和p使用r语言

In [3]:
#创建一个空的数据框
count_all = pd.DataFrame()
#遍历file_list，读取文件，保存到count_all中,文件名作为索引
file_list = os.listdir("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final\\12\\type\\count\\finaltype")
for i in file_list:
    file = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final\\12\\type\\count\\finaltype\\"+i,sep=",",index_col=0)
    #将文件名设置为多级索引
    #删除i包含的sum.txt
    i = "SGI75460"+i.replace("sum.txt","")
    file.index = pd.MultiIndex.from_product([[i],file.index])
    #将file添加到count_all中
    count_all = pd.concat([count_all,file],axis=0)
count_all
    

rf:fr_r        rf:fr_p  rf:star_r      rf:star_p  \
SGI75460067 Mt_rRNA        NaN            NaN        NaN            NaN   
            Mt_tRNA   0.996808   1.957649e-23   0.969862   9.852192e-14   
            lncRNA    0.783127   0.000000e+00   0.538432   0.000000e+00   
            miRNA     0.998310   0.000000e+00   0.907064   0.000000e+00   
            misc_RNA  0.998085   3.079559e-07   0.926256   2.725180e-03   
...                        ...            ...        ...            ...   
SGI75460105 rRNA      0.989563   2.954999e-24  -0.062730   7.464893e-01   
            scRNA     0.926631   7.336945e-02   0.900120   9.988017e-02   
            snRNA     0.954443   1.438482e-78   0.039936   6.298692e-01   
            snoRNA    0.833353  4.752912e-321   0.810630  4.998569e-290   
            tRNA      0.999922   0.000000e+00   0.997885   0.000000e+00   

                      fr:star_r     fr:star_p  
SGI75460067 Mt_rRNA         NaN           NaN  
            Mt_tRNA    0.952406  8.839637e-12  
            lncRNA     0.855258  0.000000e+00  
            miRNA      0.911561  0.000000e+00  
            misc_RNA   0.947488  1.179699e-03  
...                         ...           ...  
SGI75460105 rRNA      -0.062678  7.466918e-01  
            scRNA      0.997426  2.574120e-03  
            snRNA      0.150911  6.712388e-02  
            snoRNA     0.943541  0.000000e+00  
            tRNA       0.998379  0.000000e+00  

[91 rows x 6 columns]

In [5]:
# 创建一个空的数据框
count_all1 = pd.DataFrame()
count_temp = pd.DataFrame()
#在多级索引中分别查找"Mt_rRNA", "Mt_tRNA", "lncRNA", "miRNA", "misc_RNA", "other", "protein_coding", "pseudo", "rRNA", "scRNA", "snRNA", "snoRNA", "tRNA"的数据
for i in ["Mt_rRNA", "Mt_tRNA", "lncRNA", "miRNA", "misc_RNA", "other", "protein_coding", "pseudo", "rRNA", "scRNA", "snRNA", "snoRNA", "tRNA"]:
    count_temp=count_all.loc[count_all.index.get_level_values(1)==i]
    # 删除count_temp中第一个索引
    count_temp = count_temp.droplevel(1)
    # count_temp设置为多级索引
    count_temp.index = pd.MultiIndex.from_product([[i],count_temp.index])
    count_all1 = pd.concat([count_all1,count_temp],axis=0)
# count_all1输出到文件中
count_all1.to_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final\\12\\type\\count\\finaltype\\count_all.txt",sep="\t")
